<a href="https://colab.research.google.com/github/MaheepChaudhary/Cat_and_Dog_Audio_Classifier/blob/master/Cat_and_Dog_Audio_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!mkdir .kaggle

import json
#In username enter your username obtained from kaggle and key obtained
token = {	"username":"amratye","key":"b7620e5da4833304f5e9ec08b2daba2e"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d mmoreaux/audio-cats-and-dogs


mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
 79% 77.0M/97.0M [00:00<00:00, 121MB/s]
100% 97.0M/97.0M [00:00<00:00, 140MB/s]


In [4]:
!unzip /content/{/content}/datasets/mmoreaux/audio-cats-and-dogs/audio-cats-and-dogs.zip

Archive:  /content/{/content}/datasets/mmoreaux/audio-cats-and-dogs/audio-cats-and-dogs.zip
  inflating: cats_dogs/cat_1.wav     
  inflating: cats_dogs/cat_10.wav    
  inflating: cats_dogs/cat_100.wav   
  inflating: cats_dogs/cat_101.wav   
  inflating: cats_dogs/cat_102.wav   
  inflating: cats_dogs/cat_103.wav   
  inflating: cats_dogs/cat_105.wav   
  inflating: cats_dogs/cat_106.wav   
  inflating: cats_dogs/cat_107.wav   
  inflating: cats_dogs/cat_108.wav   
  inflating: cats_dogs/cat_109.wav   
  inflating: cats_dogs/cat_11.wav    
  inflating: cats_dogs/cat_110.wav   
  inflating: cats_dogs/cat_112.wav   
  inflating: cats_dogs/cat_113.wav   
  inflating: cats_dogs/cat_114.wav   
  inflating: cats_dogs/cat_115.wav   
  inflating: cats_dogs/cat_116.wav   
  inflating: cats_dogs/cat_117.wav   
  inflating: cats_dogs/cat_118.wav   
  inflating: cats_dogs/cat_119.wav   
  inflating: cats_dogs/cat_12.wav    
  inflating: cats_dogs/cat_120.wav   
  inflating: cats_dogs/cat_121.wav

In [41]:
#importing all libraries
import os
import IPython.display as ipd
import librosa
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Lambda,Input
from tensorflow.keras.activations import softmax,relu
from tensorflow.keras import backend as K
import numpy as np

In [82]:
#preocessing the dataset
def preprocess(path):
  data_dog = np.array([((librosa.load(os.path.join(os.path.join(path,'dog'),x)))[0],1) for x in os.listdir(os.path.join(path,'dog'))])
  data_cat = np.array([((librosa.load(os.path.join(os.path.join(path,'cat'),x)))[0],0) for x in os.listdir(os.path.join(path,'cat'))])   
  data = []
  data.extend(data_dog[:,0])
  data.extend(data_cat)
  scaler = MinMaxScaler(feature_range = (0,1))
  data = np.array(data)
  data[:,0,:] = scaler.fit_transform(data[:,0,:])
  np.random.shuffle(data)
  return data

In [83]:
train_path = '/content/cats_dogs/train'
train_x,train_y = train[:,0],train[:,1]

In [84]:
print(train_x)
print(train_x.shape)

[array([-2.8335227e-05, -8.3250152e-06, -6.8920458e-06, ...,
        3.0228417e-05,  2.1466069e-06,  0.0000000e+00], dtype=float32)
 array([0.        , 0.        , 0.        , ..., 0.11897201, 0.12529592,
       0.        ], dtype=float32)
 array([ 0.00381674,  0.00407222,  0.00156718, ..., -0.0146927 ,
       -0.01779636, -0.01206966], dtype=float32)
 array([-3.5646290e-04, -4.8690024e-04, -3.6307846e-04, ...,
       -4.5172431e-04,  2.7346627e-05, -9.4511786e-05], dtype=float32)
 array([ 0.00074632,  0.0008831 ,  0.00078632, ..., -0.01294567,
       -0.01479647,  0.        ], dtype=float32)
 array([ 0.00311164,  0.00357655,  0.00276696, ..., -0.7000312 ,
       -0.7260622 , -0.4324827 ], dtype=float32)
 array([0.00173815, 0.00318986, 0.00847096, ..., 0.00035272, 0.0010674 ,
       0.00048959], dtype=float32)
 array([0.00764209, 0.00998313, 0.00956862, ..., 0.00862761, 0.00063177,
       0.        ], dtype=float32)
 array([0.00293484, 0.0041082 , 0.00488814, ..., 0.01964642, 0.0226275

In [77]:
print(train_y)
print(train_y.shape)

[1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0
 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 0 0 1
 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1 0 0 1 1 0 0 1 1 1
 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0
 1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
(210,)


In [67]:
|train_path = '/content/cats_dogs/train'
test_path = '/content/cats_dogs/test'
train = preprocess(train_path)
test = preprocess(test_path)  
train_x,train_y = train[0],train[1]
test_x,test_y = test[0],test[1]

NameError: ignored

In [9]:
#preparing the base model
def base_model():
  model = Sequential()
  model.add(Dense(input_shape,activation = 'relu'))
  model.add(Dense(input_shape,activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(input_shape,activation = 'relu'))
  model.add(Dropout(0.3))
  model.add(Dense(input_shape,activation = 'relu'))
  #model.add(Dropout(0.5))
  #model.add(Dense(input_shape,activation = 'relu'))
  return model
  

In [ ]:
input1 = 